In [ ]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - Dual Quasars/Candidates from quasar lens searches
# Author: R. W. Pfeifle
# Creation Date: 13 January 2022
# Last Revision: 28 June 2023

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [57]:
# Inada+2008

# J093207.15+072251.3 --> Redshift in first column 1.993 does not match exactly what is shown in the last column
# --> 1.994
# J100859.55+035104.4 --> redshift in first column 1.746 does not match exactly what is shown in the last column
# --> 1.745

# Inada+2010
#for table 2 first

# We need to go back and include the inconclusive cases


inada2008t2 = pd.read_csv('Tables/Inada2008/Inada2008_t2.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
inada2008t2 = inada2008t2[inada2008t2['Comment']!='No lens object']

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

inada2008t2['Name1'] = inada2008t2['SDSS']
inada2008t2['Name2'] = inada2008t2['Name1']
#inada2008t2['z2'] = inada2008t2['z']
inada2008t2['z1_type'] = "spec"
inada2008t2['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(inada2008t2,inada2008t2['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = inada2008t2['RA'], dec = inada2008t2['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#inada2008t2['RA1_deg'] = coordconvert.ra.degree
#inada2008t2['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#inada2008t2['RA2'] = inada2008t2['RA']
#inada2008t2['Dec2'] = inada2008t2['Dec']
#
#inada2008t2['RA2_deg'] = inada2008t2['RA1_deg']
#inada2008t2['Dec2_deg'] = inada2008t2['Dec1_deg']

# Adding details about the coordinates
inada2008t2['Equinox'] = "J2000"
inada2008t2['Coordinate_waveband'] = "Optical"
inada2008t2['Coordinate_Source'] = "SDSS"

inada2008t2['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2008t2['Brightness1'] = -100
inada2008t2['Brightness_band1'] = -100
inada2008t2['Brightness_type1'] = -100

inada2008t2['Brightness2'] = -100
inada2008t2['Brightness_band2'] = -100
inada2008t2['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#inada2008t2['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#inada2008t2['Sep(kpc)'] = inada2008t2['Sep']*((cosmo.arcsec_per_kpc_proper(inada2008t2['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
inada2008t2['delta_z'] = inada2008t2['z1']-inada2008t2['z2']
inada2008t2['dV'] = (2.99e+5)*((1+inada2008t2['z1'])**2 - (1+inada2008t2['z2'])**2)/((1+inada2008t2['z1'])**2+(1+inada2008t2['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#inada2008t2['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
inada2008t2['Confirmation Method'] = "-99"
inada2008t2['Paper(s)'] = "Inada+2008"
inada2008t2['BibCode(s)'] = "2008AJ....135..496I"
inada2008t2['DOI(s)'] = "https://doi.org/10.1088/0004-6256/135/2/496"

inada2008t2['Notes'] = ''

# And dropping any columns that we don't need....
#inada2008t2.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#inada2008t2 = inada2008t2[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#inada2008t2.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

inada2008t2



,SDSS,z1,i_cor,theta_SDSS,theta_iband,Comment,Ref,z2,Name1,Name2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,SDSSJ034801.20-070416.9,1.959,18.10,1.54,1.54,No lensing object,NaN,NaN,SDSSJ034801.20-070416.9,SDSSJ034801.20-070416.9,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
1,SDSSJ072843.03+370834.9,1.404,18.97,1.34,0.08,No lensing object,NaN,NaN,SDSSJ072843.03+370834.9,SDSSJ072843.03+370834.9,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
2,SDSSJ074352.62+245743.6,2.166,19.01,1.02,1.14,No lensing object,NaN,NaN,SDSSJ074352.62+245743.6,SDSSJ074352.62+245743.6,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
3,SDSSJ083530.89+054240.7,1.696,18.50,0.99,0.18,No lensing object,NaN,NaN,SDSSJ083530.89+054240.7,SDSSJ083530.89+054240.7,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
4,SDSSJ084710.40-001302.6,0.627,18.60,0.95,0.43,QSO pair,0.626,NaN,SDSSJ084710.40-001302.6,SDSSJ084710.40-001302.6,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
5,SDSSJ085122.37+472249.0,0.894,18.80,1.47,0.03,No lensing object,NaN,NaN,SDSSJ085122.37+472249.0,SDSSJ085122.37+472249.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
6,SDSSJ093207.15+072251.3,1.993,18.96,1.42,1.95,QSO pair,1.994,NaN,SDSSJ093207.15+072251.3,SDSSJ093207.15+072251.3,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
7,SDSSJ095324.39+570319.5,0.619,18.81,2.57,0.14,No lensing object,NaN,NaN,SDSSJ095324.39+570319.5,SDSSJ095324.39+570319.5,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
8,SDSSJ100229.46+444942.7,2.052,18.39,0.72,0.25,"theta=0.8""",NaN,NaN,SDSSJ100229.46+444942.7,SDSSJ100229.46+444942.7,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
9,SDSSJ100327.37+595804.0,1.138,17.71,1.19,0.09,No multiple point sources,NaN,NaN,SDSSJ100327.37+595804.0,SDSSJ100327.37+595804.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,


In [60]:
# Inada+2008

# J093207.15+072251.3 --> Redshift in first column 1.993 does not match exactly what is shown in the last column
# --> 1.994
# J100859.55+035104.4 --> redshift in first column 1.746 does not match exactly what is shown in the last column
# --> 1.745

# This set of objects is excluded since it presumably makes up a triple image:
#SDSSJ161953.24+351321.8,1.901,18.64,,,
#SDSSJ161953.45+351323.5,,19.50, 3.13,
#No lensing object,SDSSJ161952.82+351315.4,,19.89, 8.20,No lensing object,

# We need to go back and include the inconclusive cases


inada2008t3 = pd.read_csv('Tables/Inada2008/Inada2008_t3.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
#inada2008t3 = inada2008t3[inada2008t3['Comment']!='No lens object']

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

#inada2008t3['Name2'] = inada2008t3['Name']
#inada2008t3['z2'] = inada2008t3['z']
inada2008t3['z1_type'] = "spec"
inada2008t3['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(inada2008t3,inada2008t3['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = inada2008t3['RA'], dec = inada2008t3['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#inada2008t3['RA1_deg'] = coordconvert.ra.degree
#inada2008t3['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#inada2008t3['RA2'] = inada2008t3['RA']
#inada2008t3['Dec2'] = inada2008t3['Dec']
#
#inada2008t3['RA2_deg'] = inada2008t3['RA1_deg']
#inada2008t3['Dec2_deg'] = inada2008t3['Dec1_deg']

# Adding details about the coordinates
inada2008t3['Equinox'] = "J2000"
inada2008t3['Coordinate_waveband'] = "Optical"
inada2008t3['Coordinate_Source'] = "SDSS"

inada2008t3['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2008t3['Brightness1'] = -100
inada2008t3['Brightness_band1'] = -100
inada2008t3['Brightness_type1'] = -100

inada2008t3['Brightness2'] = -100
inada2008t3['Brightness_band2'] = -100
inada2008t3['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#inada2008t3['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#inada2008t3['Sep(kpc)'] = inada2008t3['Sep']*((cosmo.arcsec_per_kpc_proper(inada2008t3['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
inada2008t3['delta_z'] = inada2008t3['z1']-inada2008t3['z2']
inada2008t3['dV'] = (2.99e+5)*((1+inada2008t3['z1'])**2 - (1+inada2008t3['z2'])**2)/((1+inada2008t3['z1'])**2+(1+inada2008t3['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#inada2008t3['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
inada2008t3['Confirmation Method'] = "-99"
inada2008t3['Paper(s)'] = "Inada+2008"
inada2008t3['BibCode(s)'] = "2008AJ....135..496I"
inada2008t3['DOI(s)'] = "https://doi.org/10.1088/0004-6256/135/2/496"

inada2008t3['Notes'] = ''

# And dropping any columns that we don't need....
#inada2008t3.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#inada2008t3 = inada2008t3[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#inada2008t3.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

inada2008t3




,Name1,z1,imag1,thetaSDSS1,Comment1,Name2,z2,imag2,thetaSDSS2,Comment2,reference,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,SDSSJ004757.25+144741.9,1.612,18.67,NaN,NaN,SDSSJ004757.87+144744.7,2.790,18.72,9.42,QSO pair,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-1.178,-106431.450115,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
1,SDSSJ025804.27-001059.9,1.551,18.93,NaN,NaN,SDSSJ025803.81-001118.1,NaN,20.05,19.43,Different SED,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
2,SDSSJ074013.44+292648.4,0.980,18.42,NaN,NaN,SDSSJ074013.42+292645.8,0.978,19.68,2.64,QSO pair,1.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.002,302.172737,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
3,SDSSJ080435.00+314311.4,2.141,19.05,NaN,NaN,SDSSJ080435.23+314328.7,NaN,19.74,17.53,Different SED,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
4,SDSSJ080658.22+273448.0,1.243,18.63,NaN,NaN,SDSSJ080658.17+273450.1,NaN,19.91,2.26,No lensing object,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
5,SDSSJ081502.51+262804.2,1.286,18.56,NaN,NaN,SDSSJ081502.28+262804.7,NaN,19.53,3.11,Different SED,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
6,SDSSJ081617.73+293639.6,0.768,18.24,NaN,NaN,SDSSJ081618.06+293643.7,NaN,19.51,5.92,No lensing object,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
7,SDSSJ081812.94+511923.4,2.098,18.98,NaN,NaN,SDSSJ081813.02+511917.5,NaN,19.80,6.05,Different SED,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
8,SDSSJ081904.56+035455.6,0.886,18.44,NaN,NaN,SDSSJ081904.40+035455.8,NaN,19.66,2.41,No lensing object,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,NaN,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,
9,SDSSJ082046.24+035742.1,1.574,18.57,NaN,NaN,SDSSJ082046.67+035740.9,1.870,19.13,6.44,QSO pair,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.296,-32418.489211,-99,Inada+2008,2008AJ....135..496I,https://doi.org/10.1088/0004-6256/135/2/496,


In [16]:
# Inada+2010
#for table 2 first

# We need to go back and include the inconclusive cases


inada2010t2 = pd.read_csv('Tables/inada2010/inada2010_t2.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
inada2010t2 = inada2010t2[inada2010t2['Comment']!='No lens object']

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

inada2010t2['Name2'] = inada2010t2['Name']
#inada2010t2['z2'] = inada2010t2['z']
inada2010t2['z1_type'] = "spec"
inada2010t2['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(inada2010t2,inada2010t2['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = inada2010t2['RA'], dec = inada2010t2['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#inada2010t2['RA1_deg'] = coordconvert.ra.degree
#inada2010t2['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#inada2010t2['RA2'] = inada2010t2['RA']
#inada2010t2['Dec2'] = inada2010t2['Dec']
#
#inada2010t2['RA2_deg'] = inada2010t2['RA1_deg']
#inada2010t2['Dec2_deg'] = inada2010t2['Dec1_deg']

# Adding details about the coordinates
inada2010t2['Equinox'] = "J2000"
inada2010t2['Coordinate_waveband'] = "Optical"
inada2010t2['Coordinate_Source'] = "SDSS"

inada2010t2['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2010t2['Brightness1'] = -100
inada2010t2['Brightness_band1'] = -100
inada2010t2['Brightness_type1'] = -100

inada2010t2['Brightness2'] = -100
inada2010t2['Brightness_band2'] = -100
inada2010t2['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#inada2010t2['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#inada2010t2['Sep(kpc)'] = inada2010t2['Sep']*((cosmo.arcsec_per_kpc_proper(inada2010t2['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
inada2010t2['delta_z'] = inada2010t2['z1']-inada2010t2['z2']
inada2010t2['dV'] = (2.99e+5)*((1+inada2010t2['z1'])**2 - (1+inada2010t2['z2'])**2)/((1+inada2010t2['z1'])**2+(1+inada2010t2['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#inada2010t2['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
inada2010t2['Confirmation Method'] = "-99"
inada2010t2['Paper(s)'] = "Inada+2010"
inada2010t2['BibCode(s)'] = "2010AJ....140..403I"
inada2010t2['DOI(s)'] = "https://doi.org/10.1088/0004-6256/140/2/403"

inada2010t2['Notes'] = ''

# And dropping any columns that we don't need....
#inada2010t2.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#inada2010t2 = inada2010t2[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#inada2010t2.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

inada2010t2




,Name,z1,i_band_psfmag(extcorr),sep_as,Comment,z2,Name2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
7,SDSS J151109.85+335701.7,0.799,18.96,1.10,FO Binary QSO,0.799,SDSS J151109.85+335701.7,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
8,SDSS J155225.62+300902.0,0.752,18.85,1.25,FO Binary QSO,0.752,SDSS J155225.62+300902.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,


In [24]:
# Inada+2010
#for table 2 first

# We need to go back and include the inconclusive cases



inada2010t3 = pd.read_csv('Tables/inada2010/inada2010_t3.csv', sep=',')

# Excluding the no lens objects because they believe these are still likely lenses
inada2010t3 = inada2010t3[inada2010t3['Comment']!='No lens object']
inada2010t3 = inada2010t3[inada2010t3['Comment']!='no lens object']
inada2010t3 = inada2010t3[inada2010t3['z2']>0]

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

#inada2010t3['Name2'] = inada2010t3['Name']
#inada2010t3['z2'] = inada2010t3['z']
inada2010t3['z1_type'] = "spec"
inada2010t3['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(inada2010t3,inada2010t3['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = inada2010t3['RA'], dec = inada2010t3['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#inada2010t3['RA1_deg'] = coordconvert.ra.degree
#inada2010t3['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#inada2010t3['RA2'] = inada2010t3['RA']
#inada2010t3['Dec2'] = inada2010t3['Dec']
#
#inada2010t3['RA2_deg'] = inada2010t3['RA1_deg']
#inada2010t3['Dec2_deg'] = inada2010t3['Dec1_deg']

# Adding details about the coordinates
inada2010t3['Equinox'] = "J2000"
inada2010t3['Coordinate_waveband'] = "Optical"
inada2010t3['Coordinate_Source'] = "SDSS"

inada2010t3['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2010t3['Brightness1'] = -100
inada2010t3['Brightness_band1'] = -100
inada2010t3['Brightness_type1'] = -100

inada2010t3['Brightness2'] = -100
inada2010t3['Brightness_band2'] = -100
inada2010t3['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#inada2010t3['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#inada2010t3['Sep(kpc)'] = inada2010t3['Sep']*((cosmo.arcsec_per_kpc_proper(inada2010t3['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
inada2010t3['delta_z'] = inada2010t3['z1']-inada2010t3['z2']
inada2010t3['dV'] = (2.99e+5)*((1+inada2010t3['z1'])**2 - (1+inada2010t3['z2'])**2)/((1+inada2010t3['z1'])**2+(1+inada2010t3['z2'])**2)
inada2010t3 = inada2010t3[np.abs(inada2010t3['dV'])<10000]
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#inada2010t3['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
inada2010t3['Confirmation Method'] = "-99"
inada2010t3['Paper(s)'] = "Inada+2010"
inada2010t3['BibCode(s)'] = "2010AJ....140..403I"
inada2010t3['DOI(s)'] = "https://doi.org/10.1088/0004-6256/140/2/403"

inada2010t3['Notes'] = ''

# And dropping any columns that we don't need....
#inada2010t3.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#inada2010t3 = inada2010t3[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#inada2010t3.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

inada2010t3





,Name1,z1,i_band_psfmag(extcorr)1,Name2,z2,i_band_psfmag(extcorr)2,Sep_as,Comment,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
39,SDSS J004018.21+005530.9,2.019,18.62,SDSS J004018.68+005525.9,2.080,19.13,8.53,QSO pair Hennawi2006,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.061,-5980.382254,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
43,SDSS J101211.29+365030.7,1.678,18.81,SDSS J101211.07+365014.3,1.678,20.01,16.60,No lens object binary QSO,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
45,SDSS J121653.10+350503.7,1.854,18.52,SDSS J121654.53+350510.8,1.800,19.33,18.82,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.054,5710.833868,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
47,SDSS J123823.37+463439.1,1.411,19.08,SDSS J123821.84+463445.8,1.457,19.97,17.15,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.046,-5650.276081,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
49,SDSS J151823.05+295925.4,1.249,18.86,SDSS J151823.43+295927.6,1.249,19.88,5.28,No lens object binary QSO,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
51,SDSS J153937.74+302023.6,1.644,18.67,SDSS J153937.10+302017.0,1.650,19.73,10.63,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.006,-677.747524,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
52,SDSS J155218.09+045635.2,1.567,18.20,SDSS J155217.94+045646.8,1.567,18.62,11.69,No lens object binary QSO,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
53,SDSS J160602.81+290048.7,0.770,18.31,SDSS J160603.02+290050.9,0.770,18.38,3.45,No lens object binary QSO Myers+2008,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
55,SDSS J163520.04+205225.1,1.775,19.03,SDSS J163519.51+205213.9,1.775,20.07,13.61,No lens object different SED binary QSO,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,
57,SDSS J165502.02+260516.5,1.892,17.64,SDSS J165501.32+260517.5,1.881,17.77,9.55,SDSS QSO different SED QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.011,1139.438094,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,


In [42]:
inada2010 = pd.concat([inada2010t2,inada2010t3])

inada2010

,Name,z1,i_band_psfmag(extcorr),sep_as,Comment,z2,Name2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes,Name1,i_band_psfmag(extcorr)1,i_band_psfmag(extcorr)2,Sep_as
7,SDSS J151109.85+335701.7,0.799,18.96,1.10,FO Binary QSO,0.799,SDSS J151109.85+335701.7,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,NaN,NaN,NaN,NaN
8,SDSS J155225.62+300902.0,0.752,18.85,1.25,FO Binary QSO,0.752,SDSS J155225.62+300902.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,NaN,NaN,NaN,NaN
43,NaN,1.678,NaN,NaN,No lens object binary QSO,1.678,SDSS J101211.07+365014.3,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,SDSS J101211.29+365030.7,18.81,20.01,16.60
49,NaN,1.249,NaN,NaN,No lens object binary QSO,1.249,SDSS J151823.43+295927.6,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,SDSS J151823.05+295925.4,18.86,19.88,5.28
51,NaN,1.644,NaN,NaN,QSO pair,1.650,SDSS J153937.10+302017.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.006,-677.747524,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,SDSS J153937.74+302023.6,18.67,19.73,10.63
52,NaN,1.567,NaN,NaN,No lens object binary QSO,1.567,SDSS J155217.94+045646.8,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,SDSS J155218.09+045635.2,18.20,18.62,11.69
53,NaN,0.770,NaN,NaN,No lens object binary QSO Myers+2008,0.770,SDSS J160603.02+290050.9,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,SDSS J160602.81+290048.7,18.31,18.38,3.45
55,NaN,1.775,NaN,NaN,No lens object different SED binary QSO,1.775,SDSS J163519.51+205213.9,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,SDSS J163520.04+205225.1,19.03,20.07,13.61
57,NaN,1.892,NaN,NaN,SDSS QSO different SED QSO pair,1.881,SDSS J165501.32+260517.5,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.011,1139.438094,-99,Inada+2010,2010AJ....140..403I,https://doi.org/10.1088/0004-6256/140/2/403,,SDSS J165502.02+260516.5,17.64,17.77,9.55


In [34]:
# Inada+2012

# We need to go back and include the inconclusive cases


inada2012t3 = pd.read_csv('Tables/Inada2012/inada2012_t3.csv', sep=',')

inada2012t3 = inada2012t3[inada2012t3['Com']!='No lens object']
inada2012t3 = inada2012t3[inada2012t3['Com']!='no lens object']
inada2012t3 = inada2012t3[inada2012t3['Com']!='separation 0.7 arcsec']
inada2012t3 = inada2012t3[inada2012t3['z2']>0]

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

inada2012t3['Name2'] = inada2012t3['Name']
#inada2012t3['z2'] = inada2012t3['z']
inada2012t3['z1_type'] = "spec"
inada2012t3['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(inada2012t3,inada2012t3['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = inada2012t3['RA'], dec = inada2012t3['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#inada2012t3['RA1_deg'] = coordconvert.ra.degree
#inada2012t3['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#inada2012t3['RA2'] = inada2012t3['RA']
#inada2012t3['Dec2'] = inada2012t3['Dec']
#
#inada2012t3['RA2_deg'] = inada2012t3['RA1_deg']
#inada2012t3['Dec2_deg'] = inada2012t3['Dec1_deg']

# Adding details about the coordinates
inada2012t3['Equinox'] = "J2000"
inada2012t3['Coordinate_waveband'] = "Optical"
inada2012t3['Coordinate_Source'] = "SDSS"

inada2012t3['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2012t3['Brightness1'] = -100
inada2012t3['Brightness_band1'] = -100
inada2012t3['Brightness_type1'] = -100

inada2012t3['Brightness2'] = -100
inada2012t3['Brightness_band2'] = -100
inada2012t3['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#inada2012t3['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#inada2012t3['Sep(kpc)'] = inada2012t3['Sep']*((cosmo.arcsec_per_kpc_proper(inada2012t3['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
inada2012t3['delta_z'] = inada2012t3['z1']-inada2012t3['z2']
inada2012t3['dV'] = (2.99e+5)*((1+inada2012t3['z1'])**2 - (1+inada2012t3['z2'])**2)/((1+inada2012t3['z1'])**2+(1+inada2012t3['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#inada2012t3['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
inada2012t3['Confirmation Method'] = "-99"
inada2012t3['Paper(s)'] = "Inada+2012"
inada2012t3['BibCode(s)'] = "2012AJ....143..119I"
inada2012t3['DOI(s)'] = "https://doi.org/10.1088/0004-6256/143/5/119"

inada2012t3['Notes'] = ''

# And dropping any columns that we don't need....
#inada2012t3.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#inada2012t3 = inada2012t3[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#inada2012t3.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

inada2012t3




,Name,z1,imag,theta,Com,z2,Name2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
5,SDSSJ124257.32+254303.0,0.827,18.71,2.77,QSO pair (z = 0.827 0.824),0.824,SDSSJ124257.32+254303.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.003,491.371895,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
11,SDSSJ142341.94+204001.5,0.842,18.97,1.06,QSO pair (z = 0.842 0.842),0.842,SDSSJ142341.94+204001.5,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,


In [39]:
# Inada+2012

# We need to go back and include the inconclusive cases


inada2012t4 = pd.read_csv('Tables/Inada2012/inada2012_t4.csv', sep=',')

inada2012t4 = inada2012t4[inada2012t4['Com']!='No lens object']
inada2012t4 = inada2012t4[inada2012t4['Com']!='no lens object']
inada2012t4 = inada2012t4[inada2012t4['z2']>0]

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

#inada2012t4['Name2'] = inada2012t4['Name']
#inada2012t4['z2'] = inada2012t4['z']
inada2012t4['z1_type'] = "spec"
inada2012t4['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(inada2012t4,inada2012t4['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = inada2012t4['RA'], dec = inada2012t4['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#inada2012t4['RA1_deg'] = coordconvert.ra.degree
#inada2012t4['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#inada2012t4['RA2'] = inada2012t4['RA']
#inada2012t4['Dec2'] = inada2012t4['Dec']
#
#inada2012t4['RA2_deg'] = inada2012t4['RA1_deg']
#inada2012t4['Dec2_deg'] = inada2012t4['Dec1_deg']

# Adding details about the coordinates
inada2012t4['Equinox'] = "J2000"
inada2012t4['Coordinate_waveband'] = "Optical"
inada2012t4['Coordinate_Source'] = "SDSS"

inada2012t4['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
inada2012t4['Brightness1'] = -100
inada2012t4['Brightness_band1'] = -100
inada2012t4['Brightness_type1'] = -100

inada2012t4['Brightness2'] = -100
inada2012t4['Brightness_band2'] = -100
inada2012t4['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#inada2012t4['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#inada2012t4['Sep(kpc)'] = inada2012t4['Sep']*((cosmo.arcsec_per_kpc_proper(inada2012t4['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
inada2012t4['delta_z'] = inada2012t4['z1']-inada2012t4['z2']
inada2012t4['dV'] = (2.99e+5)*((1+inada2012t4['z1'])**2 - (1+inada2012t4['z2'])**2)/((1+inada2012t4['z1'])**2+(1+inada2012t4['z2'])**2)
inada2012t4 = inada2012t4[np.abs(inada2012t4['dV'])<3000]
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#inada2012t4['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
inada2012t4['Confirmation Method'] = "-99"
inada2012t4['Paper(s)'] = "Inada+2012"
inada2012t4['BibCode(s)'] = "2012AJ....143..119I"
inada2012t4['DOI(s)'] = "https://doi.org/10.1088/0004-6256/143/5/119"

inada2012t4['Notes'] = ''

# And dropping any columns that we don't need....
#inada2012t4.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#inada2012t4 = inada2012t4[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#inada2012t4.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

inada2012t4





,Name,z1,imag1,Name2,z2,imag2,theta,Com,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
2,SDSSJ082234.33+170935.9,1.780,18.39,SDSSJ082234.38+170955.0,1.798,19.65,19.02,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.018,-1929.703821,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
7,SDSSJ091808.86+243550.0,1.218,18.52,SDSSJ091809.07+243604.0,1.223,19.60,14.22,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.005,-673.270932,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
9,SDSSJ094234.97+231031.1,1.833,18.99,SDSSJ094235.04+231028.9,1.833,19.70,2.45,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
24,SDSSJ125022.32+174144.5,1.246,19.06,SDSSJ125023.29+174142.3,1.241,18.63,13.91,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.005,666.368684,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
40,SDSSJ135809.86+232610.0,1.555,18.91,SDSSJ135810.68+232604.5,1.543,19.91,12.46,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.012,1407.603029,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
41,SDSSJ140012.27+232346.7,1.877,18.34,SDSSJ140012.86+232351.9,1.867,19.27,9.50,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.010,1041.083196,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
49,SDSSJ143002.88+071411.3,1.258,19.01,SDSSJ143002.66+071415.6,1.246,19.68,5.41,QSO pair (Myers+2008),spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.012,1593.239134,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,
51,SDSSJ143350.94+145008.1,1.506,18.82,SDSSJ143351.09+145005.6,1.506,19.19,3.34,QSO pair,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,


In [41]:
inada2012 = pd.concat([inada2012t3,inada2012t4])

inada2012

,Name,z1,imag,theta,Com,z2,Name2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes,imag1,imag2
5,SDSSJ124257.32+254303.0,0.827,18.71,2.77,QSO pair (z = 0.827 0.824),0.824,SDSSJ124257.32+254303.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.003,491.371895,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,NaN,NaN
11,SDSSJ142341.94+204001.5,0.842,18.97,1.06,QSO pair (z = 0.842 0.842),0.842,SDSSJ142341.94+204001.5,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,NaN,NaN
2,SDSSJ082234.33+170935.9,1.780,NaN,19.02,QSO pair,1.798,SDSSJ082234.38+170955.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.018,-1929.703821,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,18.39,19.65
7,SDSSJ091808.86+243550.0,1.218,NaN,14.22,QSO pair,1.223,SDSSJ091809.07+243604.0,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.005,-673.270932,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,18.52,19.60
9,SDSSJ094234.97+231031.1,1.833,NaN,2.45,QSO pair,1.833,SDSSJ094235.04+231028.9,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,18.99,19.70
24,SDSSJ125022.32+174144.5,1.246,NaN,13.91,QSO pair,1.241,SDSSJ125023.29+174142.3,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.005,666.368684,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,19.06,18.63
40,SDSSJ135809.86+232610.0,1.555,NaN,12.46,QSO pair,1.543,SDSSJ135810.68+232604.5,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.012,1407.603029,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,18.91,19.91
41,SDSSJ140012.27+232346.7,1.877,NaN,9.50,QSO pair,1.867,SDSSJ140012.86+232351.9,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.010,1041.083196,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,18.34,19.27
49,SDSSJ143002.88+071411.3,1.258,NaN,5.41,QSO pair (Myers+2008),1.246,SDSSJ143002.66+071415.6,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.012,1593.239134,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,19.01,19.68
51,SDSSJ143350.94+145008.1,1.506,NaN,3.34,QSO pair,1.506,SDSSJ143351.09+145005.6,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Inada+2012,2012AJ....143..119I,https://doi.org/10.1088/0004-6256/143/5/119,,18.82,19.19


In [ ]:
# Here we're adding in the information from Rusu+2013
# SDSS J132059.17+164402.59 and SDSS J132059.73+164405.6 (SDSS J1320+1644)

rusu2013 = ['SDSSJ132059.17+164402.59','SDSSJ132059.73+164405.6'] # 
for index, row in the_whills.iterrows():
    if row['Name'] in rusu2013:
        the_whills.at[index, 'Paper(s)'] += ' ; Rusu+2013 '
        the_whills.at[index, 'BibCode(s)'] += ' ; 2013ApJ...765..139R' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/765/2/139'
        the_whills.at[index, 'Notes']=' Rusu+2013 find this system comprises two QSO images and two galaxies between in a cross-like configuration. Rusu+ argue that this is most likely a lens rather than a QSO pair. All observed differences between the spectra can be attributed to a combination of extinction, microlensing,and intrinsic variability. They cannot indisputably rule out the binary hypothesis however.'
        the_whills.at[index, 'System Type'] = 'Lens / Binary Quasar'



In [45]:
# More+2016 here

#Table 1. Summary of spectroscopic observations. The ‘Selection’ column indicates the method(s) with ‘M’ for morphological selection, ‘C’ for colour
#selection, and ‘S’ for spectroscopic selection (see the text for details). Several candidates were selected by multiple methods. ‘Observation’ indicates the
#telescope (S-Subaru or K-Keck), the date and setup of the spectroscopic observations (see also Section 3.1). ‘Result’ indicates the conclusion from the
#follow-up spectroscopy as well as imaging observations described in Section 3.2. The zQSO are taken from the BOSS DR12 catalogue which has redshifts
#corrected after visual inspection of the spectra. The last column has redshifts from the follow-up spectra except when marked witha which are taken from the
#BOSS spectrum.

#Table 7. Summary of confirmed quasar pairs. See Table 1 for the RA and Dec. of the brighter quasar image (A). The
#positions and the i-band magnitudes in this table are from the SDSS data base except for J0818+0601 where we used the
#astrometrically calibrated SOAR images.

# We need to go back and include the inconclusive cases



more2016 = pd.read_csv('Tables/more2016/more2016.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.
# No separations are provided

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

#more2016['Name2'] = more2016['Name']
#more2016['z2'] = more2016['z']
more2016['z1_type'] = "spec"
more2016['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(more2016,more2016['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = more2016['RA'], dec = more2016['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#more2016['RA1_deg'] = coordconvert.ra.degree
#more2016['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#more2016['RA2'] = more2016['RA']
#more2016['Dec2'] = more2016['Dec']
#
#more2016['RA2_deg'] = more2016['RA1_deg']
#more2016['Dec2_deg'] = more2016['Dec1_deg']

# Adding details about the coordinates
more2016['Equinox'] = "J2000"
more2016['Coordinate_waveband'] = "Optical"
more2016['Coordinate_Source'] = "SDSS"

more2016['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
more2016['Brightness1'] = -100
more2016['Brightness_band1'] = -100
more2016['Brightness_type1'] = -100

more2016['Brightness2'] = -100
more2016['Brightness_band2'] = -100
more2016['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#more2016['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#more2016['Sep(kpc)'] = more2016['Sep']*((cosmo.arcsec_per_kpc_proper(more2016['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
more2016['delta_z'] = more2016['z1']-more2016['z2']
more2016['dV'] = (2.99e+5)*((1+more2016['z1'])**2 - (1+more2016['z2'])**2)/((1+more2016['z1'])**2+(1+more2016['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#more2016['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
more2016['Confirmation Method'] = "-99"
more2016['Paper(s)'] = "More+2016"
more2016['BibCode(s)'] = "2016MNRAS.456.1595M"
more2016['DOI(s)'] = "https://doi.org/10.1093/mnras/stv2813"
more2016['Notes'] = ''

# And dropping any columns that we don't need....
#more2016.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#more2016 = more2016[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#more2016.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

more2016



,Name1,RA1,Dec1,Selection,Result,Name2,RA2,Dec2,z1,z2,Sep_as,iA,iB,dV,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,SDSS J0818+0601A,08:18:30.46,+06:01:38.0,M,Quasar pair,SDSS J0818+0601B,08:18:30.420,+06:01:37.860,2.359,2.361,1.1,18.1,20.0,-177.976175,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.002,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
1,SDSS J0821+0735A,08:21:43.36,+07:35:45.9,C,Quasar pair,SDSS J0821+0735B,08:21:43.241,+07:35:45.108,2.383,2.387,1.3,20.7,21.6,-353.323363,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.004,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
2,SDSS J0930+4614A,09:30:21.16,+46:14:22.8,M,Quasar pair,SDSS J0930+4614B,09:30:20.986,+46:14:23.260,2.393,2.394,1.5,18.4,19.9,-88.109619,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.001,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
3,SDSS J1043+4320A,10:43:24.87,+43:20:49.4,M,Quasar pair,SDSS J1043+4320B,10:43:25.025,+43:20:48.958,2.245,2.225,1.7,21.4,20.7,1848.514021,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.020,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
4,SDSS J1124+5710A,11:24:55.24,+57:10:56.5,C,Quasar pair,SDSS J1124+5710B,11:24:55.440,+57:10:58.120,2.309,2.315,2.0,18.6,19.7,-541.666222,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.006,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
5,SDSS J1309+5617A,13:09:27.55,+56:17:38.9,C,Quasar pair,SDSS J1309+5617B,13:09:27.338,+56:17:41.161,2.513,2.515,2.8,20.5,21.6,-170.176423,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.002,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
6,SDSS J1405+1350A,14:05:56.92,+13:50:38.3,C,Quasar pair,SDSS J1405+1350B,14:05:56.865,+13:50:39.827,2.345,2.361,1.7,21.7,22.0,-1426.773864,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.016,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
7,SDSS J0035+2659,00:35:31.98,+26:59:59.9,M,2.294,Different SED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
8,SDSS J0139+1908,01:39:40.69,+19:08:40.7,MC,3.095,Different SED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,
9,SDSS J0206+0440,02:06:49.50,+04:40:19.0,C,2.396,Different SED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,NaN,-99,More+2016,2016MNRAS.456.1595M,https://doi.org/10.1093/mnras/stv2813,


In [47]:
# Eftekharzadeh+2017 here
# # We need to go back and include those the inconclusive stuff

# Sandrinelli+2018 has some notes on an a=object selected in this sample


#Table 1. Summary of spectroscopic observations. The ‘Selection’ column indicates the method(s) with ‘M’ for morphological selection, ‘C’ for colour
#selection, and ‘S’ for spectroscopic selection (see the text for details). Several candidates were selected by multiple methods. ‘Observation’ indicates the
#telescope (S-Subaru or K-Keck), the date and setup of the spectroscopic observations (see also Section 3.1). ‘Result’ indicates the conclusion from the
#follow-up spectroscopy as well as imaging observations described in Section 3.2. The zQSO are taken from the BOSS DR12 catalogue which has redshifts
#corrected after visual inspection of the spectra. The last column has redshifts from the follow-up spectra except when marked witha which are taken from the
#BOSS spectrum.

#Table 7. Summary of confirmed quasar pairs. See Table 1 for the RA and Dec. of the brighter quasar image (A). The
#positions and the i-band magnitudes in this table are from the SDSS data base except for J0818+0601 where we used the
#astrometrically calibrated SOAR images.


eftek2017 = pd.read_csv('Tables/Eftekharzadeh2017/Eftekharzadeh2017.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.
# No separations are provided

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

#eftek2017['Name2'] = eftek2017['Name']
#eftek2017['z2'] = eftek2017['z']
eftek2017['z1_type'] = "spec"
eftek2017['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(eftek2017,eftek2017['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = eftek2017['RA'], dec = eftek2017['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#eftek2017['RA1_deg'] = coordconvert.ra.degree
#eftek2017['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#eftek2017['RA2'] = eftek2017['RA']
#eftek2017['Dec2'] = eftek2017['Dec']
#
#eftek2017['RA2_deg'] = eftek2017['RA1_deg']
#eftek2017['Dec2_deg'] = eftek2017['Dec1_deg']

# Adding details about the coordinates
eftek2017['Equinox'] = "J2000"
eftek2017['Coordinate_waveband'] = "Optical"
eftek2017['Coordinate_Source'] = "SDSS"

eftek2017['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
eftek2017['Brightness1'] = -100
eftek2017['Brightness_band1'] = -100
eftek2017['Brightness_type1'] = -100

eftek2017['Brightness2'] = -100
eftek2017['Brightness_band2'] = -100
eftek2017['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#eftek2017['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#eftek2017['Sep(kpc)'] = eftek2017['Sep']*((cosmo.arcsec_per_kpc_proper(eftek2017['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
eftek2017['delta_z'] = eftek2017['z1']-eftek2017['z2']
eftek2017['dV'] = (2.99e+5)*((1+eftek2017['z1'])**2 - (1+eftek2017['z2'])**2)/((1+eftek2017['z1'])**2+(1+eftek2017['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#eftek2017['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
eftek2017['Confirmation Method'] = "-99"
eftek2017['Paper(s)'] = "Eftekharzadeh+2017"
eftek2017['BibCode(s)'] = "2017MNRAS.468...77E"
eftek2017['DOI(s)'] = "https://doi.org/10.1093/mnras/stx412"
eftek2017['Notes'] = ''

# And dropping any columns that we don't need....
#eftek2017.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#eftek2017 = eftek2017[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#eftek2017.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

eftek2017




,Sep_as1,Obs_stat1,RAdeg1,DEdeg1,imag1,gmag1,Class1,z1,QQ1?,Sep_as2,Obs_stat2,RAdeg2,DEdeg2,imag2,gmag2,Class2,z2,QQ2?,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,1.449,2,184.19140,35.49488,19.88,20.40,Q,-1.000,4.000,1.449,2.00000,184.19190,35.49486,19.08,19.39,Q,2.013,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-3.013,-299000.000000,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
1,1.693,2,204.77974,13.17768,18.91,19.03,Q,2.241,4.000,1.693,2.00000,204.78015,13.17741,18.87,18.96,Q,2.237,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.004,369.249628,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
2,1.999,2,228.91032,15.19300,18.37,18.70,Q,2.052,4.000,1.999,2.00000,228.91080,15.19331,18.05,18.16,Q,2.054,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.002,-195.872891,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
3,2.453,2,145.64575,23.17533,19.76,19.91,Q,1.833,3.000,2.453,2.00000,145.64598,23.17468,19.70,19.80,Q,1.833,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.000,0.000000,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
4,2.654,2,158.83012,7.88232,20.16,20.62,Q,1.218,3.000,2.654,2.00000,158.83069,7.88278,19.02,19.10,Q,1.215,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.003,404.691897,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
5,2.829,1,182.49029,11.61649,20.46,20.76,Q,0.899,3.000,2.829,1.00000,182.49049,11.61573,20.40,20.65,Q,0.904,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.005,-786.220045,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
6,2.903,2,227.17583,33.46739,20.60,20.44,Q,0.877,3.000,2.903,2.00000,227.17590,33.46820,20.56,20.38,Q,0.878,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.001,-159.254316,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
7,2.912,1,152.07303,17.25558,20.27,20.67,Q,1.087,3.000,2.912,1.00000,152.07367,17.25506,20.19,20.52,Q,1.083,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.004,573.620575,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
8,3.118,1,204.30452,60.20269,19.70,20.05,Q,1.721,3.000,3.118,1.00000,204.30472,60.20183,18.34,18.55,Q,1.730,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.009,-987.339081,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,
9,3.237,1,179.71198,13.92718,20.85,20.78,Q,2.062,3.000,3.237,1.00000,179.71272,13.92666,20.78,20.66,Q,2.080,NaN,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-0.018,-1752.508556,-99,Eftekharzadeh+2017,2017MNRAS.468...77E,https://doi.org/10.1093/mnras/stx412,


In [9]:
# Agnello+2018 here

# S0332−0021 is listed as a contaminating QSO + QSO but only one redshift is listed.
# We need to reach out and ask if this is a projected pair or not

agnello2018 = pd.read_csv('Tables/Agnello2018/agnello2018.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.
# No separations are provided
# A0326 overlaps with Schecter. They also claim there is more overlap between Schecter and their NIQs but 
# I cannot find any other matching objects to support this notion

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

agnello2018['Name2'] = agnello2018['Name']
agnello2018['z2'] = agnello2018['z']
agnello2018['z1_type'] = "spec"
agnello2018['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(agnello2018,agnello2018['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = agnello2018['RA'], dec = agnello2018['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#agnello2018['RA1_deg'] = coordconvert.ra.degree
#agnello2018['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#agnello2018['RA2'] = agnello2018['RA']
#agnello2018['Dec2'] = agnello2018['Dec']
#
#agnello2018['RA2_deg'] = agnello2018['RA1_deg']
#agnello2018['Dec2_deg'] = agnello2018['Dec1_deg']

# Adding details about the coordinates
agnello2018['Equinox'] = "J2000"
agnello2018['Coordinate_waveband'] = "Optical"
agnello2018['Coordinate_Source'] = "SDSS"

agnello2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
agnello2018['Brightness1'] = -100
agnello2018['Brightness_band1'] = -100
agnello2018['Brightness_type1'] = -100

agnello2018['Brightness2'] = -100
agnello2018['Brightness_band2'] = -100
agnello2018['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#agnello2018['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#agnello2018['Sep(kpc)'] = agnello2018['Sep']*((cosmo.arcsec_per_kpc_proper(agnello2018['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
agnello2018['delta_z'] = agnello2018['z']-agnello2018['z2']
agnello2018['dV'] = (2.99e+5)*((1+agnello2018['z'])**2 - (1+agnello2018['z2'])**2)/((1+agnello2018['z'])**2+(1+agnello2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#agnello2018['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
agnello2018['Confirmation Method'] = "-99"
agnello2018['Paper(s)'] = "Agnello+2018"
agnello2018['BibCode(s)'] = "2018MNRAS.475.2086A"
agnello2018['DOI(s)'] = "https://doi.org/10.1093/mnras/stx3226"

agnello2018['Notes'] = 'Nearly identical quasars and no lens galaxy detected. For S0332 need to verify that this is not a dual quasar.'

# And dropping any columns that we don't need....
#agnello2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#agnello2018 = agnello2018[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#agnello2018.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

agnello2018



,Name,RA(J2000),Dec(J2000),mag_i,z,Name2,z2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,A1132-0730,173.030909,-7.511781,18.31,1.990,A1132-0730,1.990,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Agnello+2018,2018MNRAS.475.2086A,https://doi.org/10.1093/mnras/stx3226,Nearly identical quasars and no lens galaxy de...
1,A1112-0335,168.180960,-3.585920,19.33,1.270,A1112-0335,1.270,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Agnello+2018,2018MNRAS.475.2086A,https://doi.org/10.1093/mnras/stx3226,Nearly identical quasars and no lens galaxy de...
2,A0326-3122(Schecter2017),51.528278,-31.381576,19.54,1.350,A0326-3122(Schecter2017),1.350,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Agnello+2018,2018MNRAS.475.2086A,https://doi.org/10.1093/mnras/stx3226,Nearly identical quasars and no lens galaxy de...
3,S1128+2402,172.077048,24.038200,17.75,1.608,S1128+2402,1.608,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Agnello+2018,2018MNRAS.475.2086A,https://doi.org/10.1093/mnras/stx3226,Nearly identical quasars and no lens galaxy de...
4,S0332-0021,53.202118,-0.365362,19.32,1.710,S0332-0021,1.710,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Agnello+2018,2018MNRAS.475.2086A,https://doi.org/10.1093/mnras/stx3226,Nearly identical quasars and no lens galaxy de...
5,A1020-1002,155.227500,-10.038888,18.14,2.030,A1020-1002,2.030,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Agnello+2018,2018MNRAS.475.2086A,https://doi.org/10.1093/mnras/stx3226,Nearly identical quasars and no lens galaxy de...


In [13]:
# Anguita+2018
# Distinct coordinates are not provided.

anguita2018 = pd.read_csv('Tables/Anguita2018/Anguita2018_t2.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.
# Verify no overlap with Schecter+2017

# 0120 was also independently identified by Ostrovski+ but I can't find a paper by Ostrovski that examines this object
# 2141 may have been looked at by Hennawi+

# 0120 and 2141 show evidence in their spectra of having distinct quasar pairs 

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

anguita2018['Name2'] = anguita2018['Name']
anguita2018['z2'] = anguita2018['z']
anguita2018['z1_type'] = "spec"
anguita2018['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(anguita2018,anguita2018['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = anguita2018['RA'], dec = anguita2018['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#anguita2018['RA1_deg'] = coordconvert.ra.degree
#anguita2018['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#anguita2018['RA2'] = anguita2018['RA']
#anguita2018['Dec2'] = anguita2018['Dec']
#
#anguita2018['RA2_deg'] = anguita2018['RA1_deg']
#anguita2018['Dec2_deg'] = anguita2018['Dec1_deg']

# Adding details about the coordinates
anguita2018['Equinox'] = "J2000"
anguita2018['Coordinate_waveband'] = "Optical"
anguita2018['Coordinate_Source'] = "SDSS"

anguita2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
anguita2018['Brightness1'] = -100
anguita2018['Brightness_band1'] = -100
anguita2018['Brightness_type1'] = -100

anguita2018['Brightness2'] = -100
anguita2018['Brightness_band2'] = -100
anguita2018['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#anguita2018['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#anguita2018['Sep(kpc)'] = anguita2018['Sep']*((cosmo.arcsec_per_kpc_proper(anguita2018['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
anguita2018['delta_z'] = anguita2018['z']-anguita2018['z2']
anguita2018['dV'] = (2.99e+5)*((1+anguita2018['z'])**2 - (1+anguita2018['z2'])**2)/((1+anguita2018['z'])**2+(1+anguita2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#anguita2018['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
anguita2018['Confirmation Method'] = "-99"
anguita2018['Paper(s)'] = "Anguita+2018"
anguita2018['BibCode(s)'] = "2018MNRAS.480.5017A"
anguita2018['DOI(s)'] = "https://doi.org/10.1093/mnras/sty2172"

anguita2018['Notes'] = 'Nearly identical quasars and no lens galaxy detected. .'

# And dropping any columns that we don't need....
#anguita2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#anguita2018 = anguita2018[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#anguita2018.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

anguita2018



,Name,g,Sep_as,z,Name2,z2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,DESJ 0058-6120,20.10,3.03,1.322,DESJ 0058-6120,1.322,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Anguita+2018,2018MNRAS.480.5017A,https://doi.org/10.1093/mnras/sty2172,Nearly identical quasars and no lens galaxy de...
1,DESJ 0120-4354,19.97,0.84,1.910,DESJ 0120-4354,1.910,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Anguita+2018,2018MNRAS.480.5017A,https://doi.org/10.1093/mnras/sty2172,Nearly identical quasars and no lens galaxy de...
2,DESJ 0544-5922,18.93,1.24,1.319,DESJ 0544-5922,1.319,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Anguita+2018,2018MNRAS.480.5017A,https://doi.org/10.1093/mnras/sty2172,Nearly identical quasars and no lens galaxy de...
3,DESJ 2141-4629,20.46,0.91,1.762,DESJ 2141-4629,1.762,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Anguita+2018,2018MNRAS.480.5017A,https://doi.org/10.1093/mnras/sty2172,Nearly identical quasars and no lens galaxy de...
4,DESJ 2200-4719,19.78,3.60,1.608,DESJ 2200-4719,1.608,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Anguita+2018,2018MNRAS.480.5017A,https://doi.org/10.1093/mnras/sty2172,Nearly identical quasars and no lens galaxy de...
5,DESJ 2250-6047,21.02,2.02,1.080,DESJ 2250-6047,1.080,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Anguita+2018,2018MNRAS.480.5017A,https://doi.org/10.1093/mnras/sty2172,Nearly identical quasars and no lens galaxy de...
6,DESJ 2337+0056,20.39,1.35,0.710,DESJ 2337+0056,0.710,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Anguita+2018,2018MNRAS.480.5017A,https://doi.org/10.1093/mnras/sty2172,Nearly identical quasars and no lens galaxy de...


In [53]:
# Spiniello+2018
# Distinct coordinates are not provided.

spiniello2018 = pd.read_csv('Tables/spiniello2018/spiniello2018.csv', sep=',')

# No distinct coordinates are provided. We need to reach out and request them.

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

spiniello2018['Name2'] = spiniello2018['Name']
#spiniello2018['z2'] = spiniello2018['z']
#spiniello2018['z1_type'] = "spec"
#spiniello2018['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(spiniello2018,spiniello2018['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = spiniello2018['RA'], dec = spiniello2018['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
spiniello2018['RA1_deg'] = coordconvert.ra.degree
spiniello2018['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
spiniello2018['RA2'] = spiniello2018['RA']
spiniello2018['Dec2'] = spiniello2018['Dec']

spiniello2018['RA2_deg'] = spiniello2018['RA1_deg']
spiniello2018['Dec2_deg'] = spiniello2018['Dec1_deg']

# Adding details about the coordinates
spiniello2018['Equinox'] = "J2000"
spiniello2018['Coordinate_waveband'] = "Optical"
spiniello2018['Coordinate_Source'] = "SDSS"

spiniello2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
spiniello2018['Brightness1'] = -100
spiniello2018['Brightness_band1'] = -100
spiniello2018['Brightness_type1'] = -100

spiniello2018['Brightness2'] = -100
spiniello2018['Brightness_band2'] = -100
spiniello2018['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#spiniello2018['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#spiniello2018['Sep(kpc)'] = spiniello2018['Sep']*((cosmo.arcsec_per_kpc_proper(spiniello2018['z']))**(-1))


## For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#spiniello2018['delta_z'] = spiniello2018['z']-spiniello2018['z2']
#spiniello2018['dV'] = (2.99e+5)*((1+spiniello2018['z'])**2 - (1+spiniello2018['z2'])**2)/((1+spiniello2018['z'])**2+(1+spiniello2018['z2'])**2)
## dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#spiniello2018['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
spiniello2018['Confirmation Method'] = "-99"
spiniello2018['Paper(s)'] = "Spiniello+2018"
spiniello2018['BibCode(s)'] = "2018MNRAS.480.1163S"
spiniello2018['DOI(s)'] = "https://doi.org/10.1093/mnras/sty1923"

spiniello2018['Notes'] = ' '

# And dropping any columns that we don't need....
#spiniello2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#spiniello2018 = spiniello2018[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#spiniello2018.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

spiniello2018



,Name,RA,Dec,Methods,Grade,Notes,Name2,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,KIDS0219-3430,02:19:36,-34:30:36,BaROQuES,2.0,,KIDS0219-3430,34.900000,-34.510000,02:19:36,-34:30:36,34.900000,-34.510000,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
1,KIDS0237-3408,02:37:20,-34:08:24,DIA,3.0,,KIDS0237-3408,39.333333,-34.140000,02:37:20,-34:08:24,39.333333,-34.140000,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
2,KIDS0256-3101,02:56:08,-31:01:09,BaROQuES,2.5,,KIDS0256-3101,44.033333,-31.019167,02:56:08,-31:01:09,44.033333,-31.019167,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
3,KIDS0313-3016,03:13:10,-30:16:26,BaROQuES,3.0,,KIDS0313-3016,48.291667,-30.273889,03:13:10,-30:16:26,48.291667,-30.273889,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
4,KIDS0324-3109,03:24:55,-31:09:18,DIA,2.0,,KIDS0324-3109,51.229167,-31.155000,03:24:55,-31:09:18,51.229167,-31.155000,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
5,KIDS0834-0139,08:34:40,-01:39:08,DIA,3.0,,KIDS0834-0139,128.666667,-1.652222,08:34:40,-01:39:08,128.666667,-1.652222,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
6,KIDS0835+0003,08:35:56,+00:03:06,DIA,2.0,,KIDS0835+0003,128.983333,0.051667,08:35:56,+00:03:06,128.983333,0.051667,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
7,KIDS0838+0124,08:38:41,+01:24:50,Multiplet,2.0,,KIDS0838+0124,129.670833,1.413889,08:38:41,+01:24:50,129.670833,1.413889,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
8,KIDS0841-0008,08:41:01,-00:08:58,DIA,2.0,,KIDS0841-0008,130.254167,-0.149444,08:41:01,-00:08:58,130.254167,-0.149444,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923
9,KIDS0847-0013,08:47:10,-00:13:03,DIA,2.5,,KIDS0847-0013,131.791667,-0.217500,08:47:10,-00:13:03,131.791667,-0.217500,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,-99,Spiniello+2018,2018MNRAS.480.1163S,https://doi.org/10.1093/mnras/sty1923


In [5]:
# Lemon+2018
# I do not include any objects classified as inconclusive
# We need to go back and include those...

# No distinct coordinates are provided by Lemon. We need to reach out and request them.

# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
lemon2018 = pd.read_csv('Tables/Lemon2018/Lemon2018_t2.csv', sep=',')
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

lemon2018['Name2'] = lemon2018['Name']
lemon2018['z2'] = lemon2018['z']
lemon2018['z1_type'] = "spec"
lemon2018['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(lemon2018,lemon2018['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = lemon2018['RA'], dec = lemon2018['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#lemon2018['RA1_deg'] = coordconvert.ra.degree
#lemon2018['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#lemon2018['RA2'] = lemon2018['RA']
#lemon2018['Dec2'] = lemon2018['Dec']
#
#lemon2018['RA2_deg'] = lemon2018['RA1_deg']
#lemon2018['Dec2_deg'] = lemon2018['Dec1_deg']

# Adding details about the coordinates
lemon2018['Equinox'] = "J2000"
lemon2018['Coordinate_waveband'] = "Optical"
lemon2018['Coordinate_Source'] = "SDSS"

lemon2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
lemon2018['Brightness1'] = -100
lemon2018['Brightness_band1'] = -100
lemon2018['Brightness_type1'] = -100

lemon2018['Brightness2'] = -100
lemon2018['Brightness_band2'] = -100
lemon2018['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#lemon2018['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#lemon2018['Sep(kpc)'] = lemon2018['Sep']*((cosmo.arcsec_per_kpc_proper(lemon2018['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
lemon2018['delta_z'] = lemon2018['z']-lemon2018['z2']
lemon2018['dV'] = (2.99e+5)*((1+lemon2018['z'])**2 - (1+lemon2018['z2'])**2)/((1+lemon2018['z'])**2+(1+lemon2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#lemon2018['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
lemon2018['Confirmation Method'] = "-99"
lemon2018['Paper(s)'] = "Lemon+2018"
lemon2018['BibCode(s)'] = "2018MNRAS.479.5060L"
lemon2018['DOI(s)'] = "https://doi.org/10.1093/mnras/sty911"

lemon2018['Notes'] = 'Nearly identical quasars and no lens galaxy detected.'

# And dropping any columns that we don't need....
#lemon2018.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#lemon2018 = lemon2018[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#lemon2018.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

lemon2018


,Name,RA,Dec,z,Sep_as,Selection,Gaia_G1,Gaia_G2,Name2,z2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J0127-1441,21.78539,-14.68861,1.76,2.96,D+WISE,20.15,20.50,J0127-1441,1.76,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
1,J0139+3526,24.88888,35.43658,0.65,2.22,D+WISE,19.46,19.65,J0139+3526,0.65,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
2,J0140+4107,25.20420,41.13331,2.50,1.44,S+WISE,17.54,NaN,J0140+4107,2.50,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
3,J0740+2926,115.05603,29.44677,0.98,2.59,D+GMM/WISE,18.20,19.64,J0740+2926,0.98,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
4,J0812+3349,123.22844,33.83062,1.49,1.99,S+GMM,19.75,NaN,J0812+3349,1.49,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
5,J1139+4143,174.94610,41.73088,2.23,2.30,D+GMM/WISE,19.46,19.62,J1139+4143,2.23,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
6,J1540+4445,235.10759,44.75457,0.61,2.74,D+GMM/WISE,19.73,20.37,J1540+4445,0.61,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
7,J1821+6005,275.37642,60.09062,2.05,1.48,S+WISE,19.96,NaN,J1821+6005,2.05,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
8,J2032-2358,308.15741,-23.97291,1.64,1.91,D+WISE,19.04,19.17,J2032-2358,1.64,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...
9,J2057+0217,314.46696,2.29683,1.52,1.06,D+GMM/WISE,20.06,20.10,J2057+0217,1.52,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2018,2018MNRAS.479.5060L,https://doi.org/10.1093/mnras/sty911,Nearly identical quasars and no lens galaxy de...


In [3]:
# Lemon+2019
# They quote 5 NIQs but I count only four in the table. Need to ask about missing one.
# I do not include any objects classified as inconclusive
# We need to go back and include those...


# The method of selection is shown for each candidate: WD, WT, WQ = ALLWISE double, triple, quad; uW = unWISE model fitting; MQ = Milliquas;

# No distinct coordinates are provided by Lemon. We need to reach out and request them.

# Lemon+2018
# I do not include any objects classified as inconclusive

# No distinct coordinates are provided by Lemon. We need to reach out and request them.
# No separations are provided. Need to ask for these. 


# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
lemon2019 = pd.read_csv('Tables/Lemon2019/Lemon2019_t4.csv', sep=',')
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

lemon2019['Name2'] = lemon2019['Name']
lemon2019['z2'] = lemon2019['z']
lemon2019['z1_type'] = "spec"
lemon2019['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(lemon2019,lemon2019['Name'])

# Converting the coordinates
#coordconvert = SkyCoord(ra = lemon2019['RA'], dec = lemon2019['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#lemon2019['RA1_deg'] = coordconvert.ra.degree
#lemon2019['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#lemon2019['RA2'] = lemon2019['RA']
#lemon2019['Dec2'] = lemon2019['Dec']
#
#lemon2019['RA2_deg'] = lemon2019['RA1_deg']
#lemon2019['Dec2_deg'] = lemon2019['Dec1_deg']

# Adding details about the coordinates
lemon2019['Equinox'] = "J2000"
lemon2019['Coordinate_waveband'] = "Optical"
lemon2019['Coordinate_Source'] = "SDSS"

lemon2019['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
lemon2019['Brightness1'] = -100
lemon2019['Brightness_band1'] = -100
lemon2019['Brightness_type1'] = -100

lemon2019['Brightness2'] = -100
lemon2019['Brightness_band2'] = -100
lemon2019['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#lemon2019['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#lemon2019['Sep(kpc)'] = lemon2019['Sep']*((cosmo.arcsec_per_kpc_proper(lemon2019['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
lemon2019['delta_z'] = lemon2019['z']-lemon2019['z2']
lemon2019['dV'] = (2.99e+5)*((1+lemon2019['z'])**2 - (1+lemon2019['z2'])**2)/((1+lemon2019['z'])**2+(1+lemon2019['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#lemon2019['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
lemon2019['Confirmation Method'] = "-99"
lemon2019['Paper(s)'] = "Lemon+2019"
lemon2019['BibCode(s)'] = "2019MNRAS.483.4242L"
lemon2019['DOI(s)'] = "https://doi.org/10.1093/mnras/sty3366"

lemon2019['Notes'] = 'Nearly identical quasars and no lens galaxy detected.'

# And dropping any columns that we don't need....
#lemon2019.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#lemon2019 = lemon2019[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#lemon2019.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']


lemon2019


,Name,RA,Dec,Selection,Gaia_G1,Gaia_G2,z,Name2,z2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J1418-1610,214.57367,-16.16897,(MQ WD uW),18.46,19.33,1.13,J1418-1610,1.13,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2019,2019MNRAS.483.4242L,https://doi.org/10.1093/mnras/sty3366,Nearly identical quasars and no lens galaxy de...
1,J1428+0500,217.23102,5.00552,(MQ WD uW),19.89,19.96,1.38,J1428+0500,1.38,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2019,2019MNRAS.483.4242L,https://doi.org/10.1093/mnras/sty3366,Nearly identical quasars and no lens galaxy de...
2,J1554+5817,238.57690,58.29635,(MQ WD uW),19.17,20.01,1.49,J1554+5817,1.49,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2019,2019MNRAS.483.4242L,https://doi.org/10.1093/mnras/sty3366,Nearly identical quasars and no lens galaxy de...
3,J2032-2358,308.15721,-23.97286,(MQ WD uW),19.12,19.26,1.64,J2032-2358,1.64,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2019,2019MNRAS.483.4242L,https://doi.org/10.1093/mnras/sty3366,Nearly identical quasars and no lens galaxy de...


In [8]:
# Now loading in the tables for Rusu+2019

rusu2019t1 = pd.read_csv('Tables/Rusu2019/Rusu2019_t1.csv', sep=',') # this contains the 91 candidates they identified
rusu2019t1b = pd.read_csv('Tables/Rusu2019/Rusu2019_t1b.csv', sep=',') # This contains previously confirmed candidate (lens? - unclear in the text) as well as rejected ones

# In table 2, confirmation_flag==2 are candidates thart have been ruled out becased on GAIA properties or spectroscopy
# confirmation_flag==1 are confirmed lenses or binary quasars

# Here we are removing those irrelevant rejected candidates:
rusu2019t1b = rusu2019t1b[rusu2019t1b['Confirmation_flag']<2]
# And now removing things mnually based on objects that were previously discovered (though we'll add Rusu+'s doi \
# info for those targets later on)


rusu2019t1['Name2'] = rusu2019t1['Name']
#rusu2019t1['z2'] = rusu2019t1['z']
#rusu2019t1['z1_type'] = "spec"
#rusu2019t1['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(rusu2019t1,rusu2019t1['Name'])

# Converting the coordinates
#coordconvert = SkyCoord(ra = rusu2019t1['RA'], dec = rusu2019t1['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
rusu2019t1['RA1_deg'] = rusu2019t1['RAdeg']#coordconvert.ra.degree
rusu2019t1['Dec1_deg'] = rusu2019t1['DEdeg']#coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#rusu2019t1['RA2'] = rusu2019t1['RA']
#rusu2019t1['Dec2'] = rusu2019t1['Dec']
#
#rusu2019t1['RA2_deg'] = rusu2019t1['RA1_deg']
#rusu2019t1['Dec2_deg'] = rusu2019t1['Dec1_deg']

# Adding details about the coordinates
rusu2019t1['Equinox'] = "J2000"
#rusu2019t1['Coordinate_waveband'] = "Optical"
#rusu2019t1['Coordinate_Source'] = "SDSS"

rusu2019t1['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
rusu2019t1['Brightness1'] = -100
rusu2019t1['Brightness_band1'] = -100
rusu2019t1['Brightness_type1'] = -100

rusu2019t1['Brightness2'] = -100
rusu2019t1['Brightness_band2'] = -100
rusu2019t1['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
rusu2019t1['Sep'] = rusu2019t1['Sep(as)'] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#rusu2019t1['Sep(kpc)'] = rusu2019t1['Sep']*((cosmo.arcsec_per_kpc_proper(rusu2019t1['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#rusu2019t1['delta_z'] = rusu2019t1['z']-rusu2019t1['z2']
#rusu2019t1['dV'] = (2.99e+5)*((1+rusu2019t1['z'])**2 - (1+rusu2019t1['z2'])**2)/((1+rusu2019t1['z'])**2+(1+rusu2019t1['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#rusu2019t1['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
rusu2019t1['Confirmation Method'] = "-99"
rusu2019t1['Paper(s)'] = "Rusu+2019"
rusu2019t1['BibCode(s)'] = "2019MNRAS.486.4987R"
rusu2019t1['DOI(s)'] = "https://doi.org/10.1093/mnras/stz1142"

#rusu2019t1['Notes'] = 'Nearly identical quasars and no lens galaxy detected.'

# And dropping any columns that we don't need....
#rusu2019t1.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#rusu2019t1 = rusu2019t1[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#rusu2019t1.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']


rusu2019t1



,Name,RAdeg,DEdeg,Num_comp,i_mag,Sep(as),Rank,Gmag1,Gmag2,Gmag3,Notes,Name2,RA1_deg,Dec1_deg,Equinox,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,000815-043634,2.061059,-4.609377,2,18.57,2.4,C,18.82,20.09,NaN,similar colour p-l; both components negligible...,000815-043634,2.061059,-4.609377,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.4,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
1,003309-120520,8.287252,-12.088925,3,18.10,6.8,C,18.93,20.38,NaN,p-l (both negligible AEN; pm; and p) + red inn...,003309-120520,8.287252,-12.088925,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,6.8,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
2,004106+032726,10.273022,3.457205,2,18.43,2.4,C,18.91,20.36,NaN,p-l; includes SDSS z=1.282 QSO; both negligibl...,004106+032726,10.273022,3.457205,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.4,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
3,004518+405433,11.325876,40.909217,2,18.69,3.1,C,19.32,18.85,NaN,similar colour p-l; includes z=1.228 QSO (Huo ...,004518+405433,11.325876,40.909217,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.1,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
4,012221+291431,20.587958,29.242069,2,18.30,2.4,C,18.41,20.72,NaN,similar colour p-l; one component negligible A...,012221+291431,20.587958,29.242069,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.4,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
5,012256+783855,20.733302,78.648546,2,18.43,2.0,C,18.94,18.99,NaN,similar colour p-l; both negligible AEN; pm; a...,012256+783855,20.733302,78.648546,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.0,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
6,012648+411136,21.698143,41.193204,2,19.13,3.1,C,19.26,20.15,NaN,similar colour p-l; both negligible AEN; pm; a...,012648+411136,21.698143,41.193204,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.1,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
7,013021+072516,22.585897,7.421231,2,18.80,2.0,C,18.98,19.65,NaN,p-l; one component negligible AEN; pm; and p; ...,013021+072516,22.585897,7.421231,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.0,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
8,014114-062740,25.307825,-6.461006,2,19.05,2.4,C,20.79,NaN,NaN,similar colour p-l; only one has Gaia data; ne...,014114-062740,25.307825,-6.461006,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.4,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
9,014455+271137,26.230638,27.193616,2,19.01,1.9,C,19.53,19.63,NaN,similar colour p-l; both negligible AEN; pm; a...,014455+271137,26.230638,27.193616,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,1.9,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142


In [9]:
rusu2019t1b

rusu2019t1b['Name2'] = rusu2019t1b['Name']
#rusu2019t1b['z2'] = rusu2019t1b['z']
#rusu2019t1b['z1_type'] = "spec"
#rusu2019t1b['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(rusu2019t1b,rusu2019t1b['Name'])

# Converting the coordinates
#coordconvert = SkyCoord(ra = rusu2019t1b['RA'], dec = rusu2019t1b['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
rusu2019t1b['RA1_deg'] = rusu2019t1b['RAdeg']#coordconvert.ra.degree
rusu2019t1b['Dec1_deg'] = rusu2019t1b['DEdeg']#coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#rusu2019t1b['RA2'] = rusu2019t1b['RA']
#rusu2019t1b['Dec2'] = rusu2019t1b['Dec']
#
#rusu2019t1b['RA2_deg'] = rusu2019t1b['RA1_deg']
#rusu2019t1b['Dec2_deg'] = rusu2019t1b['Dec1_deg']

# Adding details about the coordinates
rusu2019t1b['Equinox'] = "J2000"
#rusu2019t1b['Coordinate_waveband'] = "Optical"
#rusu2019t1b['Coordinate_Source'] = "SDSS"

rusu2019t1b['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
rusu2019t1b['Brightness1'] = -100
rusu2019t1b['Brightness_band1'] = -100
rusu2019t1b['Brightness_type1'] = -100

rusu2019t1b['Brightness2'] = -100
rusu2019t1b['Brightness_band2'] = -100
rusu2019t1b['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
rusu2019t1b['Sep'] = rusu2019t1b['Sep(as)'] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#rusu2019t1b['Sep(kpc)'] = rusu2019t1b['Sep']*((cosmo.arcsec_per_kpc_proper(rusu2019t1b['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#rusu2019t1b['delta_z'] = rusu2019t1b['z']-rusu2019t1b['z2']
#rusu2019t1b['dV'] = (2.99e+5)*((1+rusu2019t1b['z'])**2 - (1+rusu2019t1b['z2'])**2)/((1+rusu2019t1b['z'])**2+(1+rusu2019t1b['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#rusu2019t1b['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
rusu2019t1b['Confirmation Method'] = "-99"
rusu2019t1b['Paper(s)'] = "Rusu+2019"
rusu2019t1b['BibCode(s)'] = "2019MNRAS.486.4987R"
rusu2019t1b['DOI(s)'] = "https://doi.org/10.1093/mnras/stz1142"

#rusu2019t1b['Notes'] = 'Nearly identical quasars and no lens galaxy detected.'

# And dropping any columns that we don't need....
#rusu2019t1b.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#rusu2019t1b = rusu2019t1b[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#rusu2019t1b.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']


rusu2019t1b




,Name,RAdeg,DEdeg,Num_comp,i_mag,Sep(as),Rank,Confirmation_flag,Notes,Name2,RA1_deg,Dec1_deg,Equinox,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,013459+243049,23.745589,24.513635,2,19.40,3.7,C,1,G=19.60; 20.17; p-l; two SDSS QSOs at z=2.093 ...,013459+243049,23.745589,24.513635,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.7,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
1,014710+463043,26.792452,46.512081,2,15.57,3.2,A,1,G=15.89; 16.18; 16.74; 18.26; similar colour p...,014710+463043,26.792452,46.512081,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.2,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
2,024526-055700,41.356685,-5.950128,2,18.67,1.7,C,1,G=19.73; 19.25; similar colour p-l + red inner...,024526-055700,41.356685,-5.950128,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,1.7,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
3,025934-233802,44.889982,-23.633792,2,19.21,2.7,B,1,G=20.34; 19.37; p-l + red inner component; PSJ...,025934-233802,44.889982,-23.633792,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.7,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
4,094235+231030,145.645825,23.175133,2,18.87,2.4,C,1,G=19.10; 19.92; similar colour p-l; z=1.83 QSO...,094235+231030,145.645825,23.175133,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.4,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
5,110633-182124,166.639282,-18.356688,2,16.95,3.1,C,1,G=17.07; 18.20; similar colour p-l; HE1104-180...,110633-182124,166.639282,-18.356688,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.1,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
6,110932+531636,167.384487,53.276552,2,18.71,3.2,C,1,G=18.97; 19.68; similar colour p-l; includes S...,110932+531636,167.384487,53.276552,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.2,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
7,113441-210323,173.668953,-21.056307,3,16.81,3.7,A,1,G=17.17; 17.19; 18.94; 17.27; four p-l sources...,113441-210323,173.668953,-21.056307,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.7,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
8,120451+442836,181.210712,44.476590,2,18.84,3.0,B,1,G=18.81; 19.65; similar colour p-l; SQLS QSO a...,120451+442836,181.210712,44.476590,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.0,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142
9,120630+433219,181.623684,43.538734,2,18.52,3.0,B,1,G=18.87; 18.84; similar colour p-l + red compo...,120630+433219,181.623684,43.538734,J2000,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.0,-99,Rusu+2019,2019MNRAS.486.4987R,https://doi.org/10.1093/mnras/stz1142


In [2]:
# Lemon+2020
#Quasar pairs. Selection: G1: Gaia 1, G2: Gaia 2, V: variability, C: component fitting. NIQ stands for nearly identical quasar pair.

# No distinct coordinates are provided by Lemon. We need to reach out and request them.


# Lemon+2018
# I do not include any objects classified as inconclusive
# We need to go back and include those...


# No distinct coordinates are provided by Lemon. We need to reach out and request them.

# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
lemon2020 = pd.read_csv('Tables/Lemon2020/Lemon2020_t2.csv', sep=',')
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

lemon2020['Name2'] = lemon2020['Name']
lemon2020['z2'] = lemon2020['z']
lemon2020['z1_type'] = "spec"
lemon2020['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(lemon2020,lemon2020['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = lemon2020['RA'], dec = lemon2020['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#lemon2020['RA1_deg'] = coordconvert.ra.degree
#lemon2020['Dec1_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#lemon2020['RA2'] = lemon2020['RA']
#lemon2020['Dec2'] = lemon2020['Dec']
#
#lemon2020['RA2_deg'] = lemon2020['RA1_deg']
#lemon2020['Dec2_deg'] = lemon2020['Dec1_deg']

# Adding details about the coordinates
lemon2020['Equinox'] = "J2000"
lemon2020['Coordinate_waveband'] = "Optical"
lemon2020['Coordinate_Source'] = "SDSS"

lemon2020['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
lemon2020['Brightness1'] = -100
lemon2020['Brightness_band1'] = -100
lemon2020['Brightness_type1'] = -100

lemon2020['Brightness2'] = -100
lemon2020['Brightness_band2'] = -100
lemon2020['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#lemon2020['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#lemon2020['Sep(kpc)'] = lemon2020['Sep']*((cosmo.arcsec_per_kpc_proper(lemon2020['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
lemon2020['delta_z'] = lemon2020['z']-lemon2020['z2']
lemon2020['dV'] = (2.99e+5)*((1+lemon2020['z'])**2 - (1+lemon2020['z2'])**2)/((1+lemon2020['z'])**2+(1+lemon2020['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
#lemon2020['Selection Method'] = "" #DPSELs
lemon2020['Confirmation Method'] = "-99"
lemon2020['Paper(s)'] = "Lemon+2020"
lemon2020['BibCode(s)'] = "2020MNRAS.494.3491L"
lemon2020['DOI(s)'] = "https://doi.org/10.1093/mnras/staa652"

lemon2020['Notes'] = 'Nearly identical quasats and no lens galaxy detected.'

# And dropping any columns that we don't need....
#lemon2020.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#lemon2020 = lemon2020[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#lemon2020.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']


lemon2020




,Name,Sep_as,RA(J2000),Dec(J2000),Selection,z,Name2,z2,z1_type,z2_type,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,DESJ0027+0232,2.72,6.7619,2.5375,G1,2.020,DESJ0027+0232,2.020,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
1,DESJ0101-4943,4.02,15.3366,-49.7234,G1,2.100,DESJ0101-4943,2.100,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
2,DESJ0118-0104,1.74,19.5501,-1.0785,G1,0.740,DESJ0118-0104,0.740,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
3,DESJ0118-3115,1.00,19.6679,-31.2619,C,1.740,DESJ0118-3115,1.740,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
4,DESJ0122+0358,1.71,20.5990,3.9771,G1,1.690,DESJ0122+0358,1.690,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
5,DESJ0229+0320,2.08,37.4924,3.3419,V,1.430,DESJ0229+0320,1.430,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
6,DESJ0254-2243,2.32,43.5720,-22.7315,G1,2.040,DESJ0254-2243,2.040,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
7,DESJ0313-2546,2.20,48.4088,-25.7751,G1,1.955,DESJ0313-2546,1.955,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
8,DESJ0443-2403,1.85,70.9802,-24.0572,G1,1.780,DESJ0443-2403,1.780,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...
9,DESJ2215-5204,2.74,333.9171,-52.0679,G1,2.350,DESJ2215-5204,2.350,spec,spec,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,-99,Lemon+2020,2020MNRAS.494.3491L,https://doi.org/10.1093/mnras/staa652,Nearly identical quasats and no lens galaxy de...


In [ ]:
# Here we're adding in information from Hutsemekers+2020 
# This is a follow-up to the object SDSS J081830.46+060138.0 (J0818+0601)

hutsemekers2020 = ['J0818+0601A'] # 
for index, row in the_whills.iterrows():
    if row['Name'] in hutsemekers2020:
        the_whills.at[index, 'Paper(s)'] += ' ; Hutsemekers+2020 '
        the_whills.at[index, 'BibCode(s)'] += ' ; 2020A&A...633A.101H' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1051/0004-6361/201936973'
        the_whills.at[index, 'Notes']='Hutsemekers+ find in their spectropolarimetric obs that the BAL profiles are nearly identical between the two spectra. This strongly suppoprts the notion of a lens rather than a binary given how unlikely it is that to find two similar absorbers in the two AGNs. Microlensing appears to show two sources of continuum. An intervening absorption line system is observed around z=1 in both images but with different intensities. This absorption line system hints at the presence of a lens galaxy.'
        the_whills.at[index, 'System Type'] = 'Lens / Binary Quasar'

